## Welcome to the quick notebook on using Llama 2 🦙 🚢

### To begin, make sure you:
1. pip install weaviate-client
2. pip install llama-index
3. pip install replicate 

### You will also need to have an access key for:
1. [OpenAI](https://openai.com/)
2. [Replicate](https://replicate.com/)

## Connect to Weaviate

In [ ]:
import weaviate 

client = weaviate.Client(
  url="https://URL.weaviate.network",  # URL to Weaviate instance
)

client.schema.get()  # Get the schema to test connection

## Create Schema

In [2]:
blog_post_schema = {
   "classes": [
       {
           "class": "BlogPost",
           "description": "Blog post from the Weaviate website.",
           "vectorizer": "text2vec-openai",
           "properties": [
               {
                  "name": "Content",
                  "dataType": ["text"],
                  "description": "Content from the blog post",
               }
            ]
        }
    ]
}

client.schema.delete_all()

client.schema.create(blog_post_schema)

print("Schema was created.")

Schema was created.


## Load data

In [3]:
from llama_index import SimpleDirectoryReader

# load the blogs in using the reader
blogs = SimpleDirectoryReader('./data').load_data()

## Construct vector store

In [6]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, ListIndex
from llama_index.storage.storage_context import StorageContext

vector_store = WeaviateVectorStore(weaviate_client=client, class_prefix="Blogs_index")

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(blogs, storage_context=storage_context)

In [14]:
from llama_index.llms import Replicate
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

# The replicate endpoint
LLAMA_13B_V2_CHAT = "a16z-infra/llama13b-v2-chat:5ec5fdadd80ace49f5a2b2178cceeb9f2f77c493b85b1131002c26e6b2b13184"

# inject custom system prompt into llama-2
def custom_completion_to_prompt(completion: str) -> str:
    return completion_to_prompt(
        completion,
        system_prompt=(
            "You are a Q&A assistant. Your goal is to answer questions as "
            "accurately as possible in the instructions and context provided."
        )
    )


llm = Replicate(
    model=LLAMA_13B_V2_CHAT,
    temperature=0,
    # override max tokens since it's interpreted
    # as context window instead of max tokens
    context_window=4096,
    # override completion representation for llama 2
    completion_to_prompt=custom_completion_to_prompt
)

# set a global service context
ctx = ServiceContext.from_defaults(llm=llm)
set_global_service_context(ctx)

## Query time 

In [15]:
query_engine = index.as_query_engine()

In [16]:
response = query_engine.query("What is ref2vec? How does this feature help with building recommendation systems?")
print(response)

 Ref2Vec is a feature in Weaviate that allows for the vectorization of data objects, including cross-references to other objects. In the context of recommendation systems, Ref2Vec can be used to represent a user's interests by drawing a graph of cross-references from the user to objects they have engaged with. This allows Weaviate to conveniently and immediately learn from each user's preferences and actions to provide improved and up-to-date characterizations.
The benefits of using Ref2Vec in recommendation systems include:
1. Capturing user interests and tendencies: By representing a user's interests and tendencies across multiple axes, such as product categories or even fashion styles, Ref2Vec can help improve the accuracy of recommendations.
2. Improved relevance ranking: By using the averaged vector of a user's cross-referenced objects as a search query in Product objects, Ref2Vec can help rank search results based on relevance to the user's interests.
3. Lightning-fast recommenda